In [549]:
import re
h_tone = "áéíóú"
l_tone = "àèìòù"
f_tone = "âêîôû"

class Autorep:   
    def __init__(self, word):
        self.word = word
        self.tone = ''
        self.mel = ''
        self.assoc = []
        pos = 0

        for i,seg in enumerate(word):
            if seg in f_tone:
                self.tone += "F"
            elif seg in h_tone:
                self.tone += "H" 
            elif seg in l_tone:
                self.tone += "L"   

        self.mel = self.tone 
        
        # if the tone sequence contains 'F', convert 'F' into 'HL'

        contour_count = 0 
        for i in range(len(self.tone)):
            self.assoc.append((i+1,i+1)) 
            if self.tone[i] == 'F':
                k,v = self.assoc[i]
                self.mel = self.mel.replace('F','HL')
                self.assoc.insert(2*i+1,(i+contour_count,i+1))


        for i in range(1,len(self.mel)):
            j,k = self.assoc[i]
            p,q = self.assoc[i-1]
            if self.mel[i] == self.mel[i-1]:
                for j in range(i, len(self.assoc)):
                    self.assoc[j] = (self.assoc[i-1][0], self.assoc[j][1])
            else:
                j = p + 1
                self.assoc[i] = (j,k)
        
        self.mel = re.sub(r"(.)\1+", r"\1", self.mel)
           
        print (self.word, self.tone, self.mel, self.assoc) 

    def _tone_pos(self,check_tone):
            check_tone = check_tone.upper()
            if check_tone not in set(self.mel):
                return False
            else:   
                return [1+ i for i, j in enumerate(self.mel) if j == check_tone]      
    
    def spreading_count(self,check_tone):
        for i in self._tone_pos(check_tone):
            spreading = 0
            for j in self.assoc:
                if j[0] == i:
                    spreading += 1 
            print(check_tone,i,"spread length",spreading)

    def __eq__(self, other):
        return self.mel == other.mel and self.assoc == other.assoc  


In [551]:
test_tone = ['ââ', 'ààââ','mámàmámààà', 'màmámá', 'màmàmá','kâkâmá', 'kâmà', 'mákâ','màkâ'] # ââ
list_of_word = [Autorep(word)for word in test_tone]

a = Autorep("ààââ")
a.spreading_count('L')

ââ FF HLHL [(1, 1), (2, 1), (3, 2), (4, 2)]
ààââ LLFF LHLHL [(1, 1), (1, 2), (2, 3), (3, 3), (4, 4), (5, 4)]
mámàmámààà HLHLLL HLHL [(1, 1), (2, 2), (3, 3), (4, 4), (4, 5), (4, 6)]
màmámá LHH LH [(1, 1), (2, 2), (2, 3)]
màmàmá LLH LH [(1, 1), (1, 2), (2, 3)]
kâkâmá FFH HLHLH [(1, 1), (2, 1), (3, 2), (4, 2), (5, 3)]
kâmà FL HL [(1, 1), (2, 1), (2, 2)]
mákâ HF HL [(1, 1), (1, 2), (2, 2)]
màkâ LF LHL [(1, 1), (2, 2), (3, 2)]
ààââ LLFF LHLHL [(1, 1), (1, 2), (2, 3), (3, 3), (4, 4), (5, 4)]
L 1 spread length 2
L 3 spread length 1
L 5 spread length 1
